Dados que são apresentados no Relatório ONE PAGE:

Data / dia da semana
Unidade

Indicadores Financeiros
- Faturamento Total (NAO COLOCAR)
- Faturamento Convenios (Top 5 Convenios)
- Faturamento Particular (Cartão e Dinheiro)
- Ticket Médio

Indicadores Médicos (verificar na aba atendimentos do asa)
- Quais médicos estavam atendendo
- Médico que mais atendeu consultas (top 3)
- Médico que mais gerou faturamento (top 3)
- Ticket Medio dos Medicos (retirar guia orçamento)
- Médico que mais gerou testes (top 3)
- Médico que mais fez procedimentos (top 3)

Indicadores Agenda
- Numero de Consultas
- Numero de Primeiras vezes (triagens, verificar "retriagens")
- Numero de testes
- Quantos estavam marcados + quantidade e % de Faltas
- Tempo Médio de espera
- Convenios mais atendidos (top 5)
- Usuarios que mais marcaram e quantas consultas foram marcadas


In [ ]:
import pandas as pd
from collections import OrderedDict
import yagmail
import pyautogui
import warnings

warnings.filterwarnings('ignore')

# FINANCEIROS --------------------------------------------------------------------------

faturamento_convenios_base = pd.read_excel('faturamento de cada convenio - valores analiticos por convenio (fat op).xls')
faturamento_convenios = faturamento_convenios_base[['Unnamed: 1','Unnamed: 9']]
faturamento_convenios.columns = ['Convenios','Faturamento']
ultima_linha = len(faturamento_convenios) - 1 
faturamento_convenios = faturamento_convenios.drop(ultima_linha)
ultima_linha = len(faturamento_convenios) - 1 

# Pegar Faturamento Bruto
faturamento_bruto = faturamento_convenios.iloc[ultima_linha, 1]

#-----------------------------------------------------

# Retirar Naan
faturamento_convenios['Convenios'] = faturamento_convenios['Convenios'].fillna('x')

faturamento_convenios['Faturamento'] = faturamento_convenios['Faturamento'].fillna('x')

#-----------------------------------------------------

faturamento_convenios = faturamento_convenios.drop(ultima_linha)

faturamento_convenios['Convenios'] = faturamento_convenios['Convenios'].astype(str)
faturamento_convenios['Faturamento'] = faturamento_convenios['Faturamento'].astype(str)

#-----------------------------------------------------

#Pegar lista com convenios atendidos no dia
lista_convenios = list(faturamento_convenios['Convenios'])

lista_convenios_nova = []
lista_convenios_nova2 = []
lista_convenios_nova3 = []
lista_convenios_nova4 = []

for x in lista_convenios:
    if x[0] != '9':
        lista_convenios_nova.append(x)
        if x != ' ':
            lista_convenios_nova2.append(x)
            if x != 'x':
                lista_convenios_nova3.append(x)
                if x != 'Código':
                    lista_convenios_nova4.append(x)
                    
lista_de_convenios = list(OrderedDict.fromkeys(lista_convenios_nova4))

#-----------------------------------------------------

faturamento_valores = faturamento_convenios_base[['Unnamed: 8','Unnamed: 9']]

faturamento_valores = faturamento_valores[faturamento_valores['Unnamed: 8'] == 'Valor :']

faturamento_valores = faturamento_valores.reset_index(drop=True)

#-----------------------------------------------------

faturamento_valores['Unnamed: 8'] = faturamento_valores['Unnamed: 8'].astype(str)
ultima_linha = len(faturamento_valores) - 1 
faturamento_valores = faturamento_valores.drop(ultima_linha)

faturamento_valores['Unnamed: 8'] = lista_de_convenios

faturamento_valores.columns = ['Convenios','Valores']

# Excluir 'Particular do dataframe'
for i, convenio in enumerate(faturamento_valores['Convenios']):
    if convenio == 'Particular':
        faturamento_valores = faturamento_valores.drop(i)
        
# Colocar na Ordem
faturamento_valores = faturamento_valores.sort_values(by=['Valores'], ascending=False)
faturamento_valores = faturamento_valores.reset_index(drop=True)


# faturamento_valores = faturamento_valores.head()

#-------------------------------------------------------

# Pegar total de Faltas, porcentagem e total de pacientes que estavam marcados
agenda = pd.read_excel('numero de pacientes marcados e faltas - ocupacao da agenda (ag ger).xls')
agenda = agenda[['Gerson Schorr Clinica de Alergia e Dermatologia','Unnamed: 7','Unnamed: 8']]
agenda = agenda.dropna(thresh=2)
agenda = agenda.reset_index(drop=True)
agenda = agenda.drop(0)
agenda.columns = ['Agendas','Faltas','Marcações']
agenda = agenda.reset_index(drop=True)
agenda['Atendimentos'] = agenda['Marcações'] - agenda['Faltas']
agenda = agenda[agenda['Marcações'] != 0]
lista_faltas = agenda['Faltas']
agenda = agenda.drop(['Faltas'], axis=1)
agenda.insert(2, 'Faltas', lista_faltas) 
agenda = agenda.reset_index(drop=True)
ultima_linha = len(agenda) - 1 
agenda = agenda.drop(ultima_linha)

agenda_consultas = agenda[agenda['Agendas'] != 'Enfermagem']
agenda_consultas = agenda_consultas[agenda_consultas['Agendas'] != 'Sala Procedimentos']
# print('\n',agenda,'\n')

# total_consultas = agenda_consultas['Atendimentos'].sum()
# total_atendimentos = agenda['Atendimentos'].sum()

total_Marcações = agenda['Marcações'].sum()
total_Faltas = agenda['Faltas'].sum()
porcent_faltas = (total_Faltas/total_Marcações)*100 

#-------------------------------------------------------

# Pegar Consultas por convenio e total de consultas
cons_por_conv = pd.read_excel('numero de consultas por convenio - valores sinteticos de proced (fat ger).xls')
cons_por_conv = cons_por_conv[['Unnamed: 1','Unnamed: 8']]
cons_por_conv = cons_por_conv.dropna()
cons_por_conv = cons_por_conv.reset_index(drop=True)
cons_por_conv = cons_por_conv.drop(0)
cons_por_conv.columns = ['Convenio', 'Consultas']

try:
    cons_por_conv['Convenio'] = cons_por_conv['Convenio'].replace('Bradesco Saúde Operadora de Planos','Bradesco Saúde')
except:
    pass

cons_por_conv = cons_por_conv.groupby(['Convenio']).sum()

cons_por_conve = list(cons_por_conv.index)
cons_por_conve = list(OrderedDict.fromkeys(cons_por_conve))
cons_por_conv = cons_por_conv.reset_index(drop=True)
cons_por_conv['Convenios'] = cons_por_conve
cons_por_conv = cons_por_conv[['Convenios','Consultas']]
cons_por_conv = cons_por_conv.sort_values(by=['Consultas'], ascending=False)
total_consultas = cons_por_conv['Consultas'].sum()

try:
    cons_por_conv = cons_por_conv[cons_por_conv['Convenios'] != 'Particular']
except:
    pass

cons_por_conv = cons_por_conv.reset_index(drop=True)
# cons_por_conv = cons_por_conv.head()


#-------------------------------------------------------

# Pegar faturamento do particular
rel_caixa = pd.read_excel('relatorio do caixa particular - movimento analitico caixa (cx op).xls')

rel_caixa = rel_caixa[(rel_caixa['Gerson Schorr Clinica de Alergia e Dermatologia'] == 'TOTAL GERAL') |
                      (rel_caixa['Gerson Schorr Clinica de Alergia e Dermatologia'] == 'Dinheiro') ]

total_din = rel_caixa.iloc[0, 4]
total_caixa = rel_caixa.iloc[1, 3]
total_cartao = total_caixa - total_din

# Pegar Faturamento com convenios e sem o particular
fat_convenios = float(faturamento_bruto) - total_caixa

# Pegar Ticket Médio
ticket_medio = faturamento_bruto/total_consultas




# MÉDICOS ------------------------------------------------------------------------------



#----------------------------------------


fat_med = pd.read_excel('faturamento de cada medico - contas por medico (fat op).XLS')

fat_med.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14']
fat_med = fat_med[['1','11']]

fat_valores = fat_med.dropna()

fat_valores['11'] = fat_valores['11'].astype(str)

lista_fat_valores = list(fat_valores['11'])

for item in lista_fat_valores:
    if "/" in item:
        lista_fat_valores.remove(item)
    
#----------------------------------------

fat_med = fat_med.dropna(subset=['1'])

lista_fat_med = list(fat_med['1'])

itens_excluir = ['<456>','Total de Atendimentos :','Gerson Schorr Clinica de Alergia e Dermatologia','Data','Empresa Gerson Schorr Clinica de Alergia e Dermatologia, Unidade Matriz, Período em 10/05/2022','Tecnoarte Informática']
itens_nova = []
medicos_fat = []

for item in lista_fat_med:
    if item not in itens_excluir:
        itens_nova.append(item)
        
for item in itens_nova:  
    if "/" not in item:
        medicos_fat.append(item)
        
medicos_fat = list(OrderedDict.fromkeys(medicos_fat))     

#----------------------------------------

fat_medicos = pd.DataFrame((zip(medicos_fat, lista_fat_valores)), columns = ['Medicos', 'Faturamento'])

fat_medicos = fat_medicos[fat_medicos['Faturamento'] != '0']

fat_medicos['Faturamento'] = fat_medicos['Faturamento'].astype(float)

fat_medicos = fat_medicos.sort_values(by=['Faturamento'],ascending=False)

fat_medicos = fat_medicos.reset_index(drop=True)

ver = fat_medicos['Faturamento'].sum()

# display(ver)

# fat_medicos = fat_medicos.head(3)

# display(medicos_consultas)
# display(fat_medicos)

# AGENDA -------------------------------------------------------------------------------

triagens = pd.read_excel('numero de consultas e triagens - proced realizado sinteticos (fat op).xls')
triagens = triagens[triagens['Unnamed: 1'] == 'TRIAGEM']
triagens.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14']
triagens = triagens.reset_index(drop=True)
triagens = triagens.iloc[0,10]
porcentriagem = (triagens/total_consultas) * 100

#---------------------------------------

mais_marcou = pd.read_excel('usuarios que mais marcaram - marcações por usuario sintetico (ag op).xls')

mais_marcou.columns = ['1','2','3','4','5','6','7','8','9','10','11','12','13','14']
qtd_consultas_marcadas = mais_marcou[mais_marcou['9'] == 'Total Geral :']
qtd_consultas_marcadas = qtd_consultas_marcadas.reset_index(drop=True)
qtd_consultas_marcadas = qtd_consultas_marcadas.loc[0, '11']

#---------------------------------------

mais_marcou = mais_marcou.dropna(thresh=3, axis=1)
mais_marcou = mais_marcou.drop([0,1,2,3])
mais_marcou = mais_marcou.reset_index(drop=True)
mais_marcou = mais_marcou.dropna()
mais_marcou = mais_marcou.drop([0,1,2,3])
mais_marcou = mais_marcou.drop(['1'], axis=1)
mais_marcou.columns = ['Usuarios','Marcações']
mais_marcou = mais_marcou.sort_values(by=['Marcações'], ascending=False)

indexNames = mais_marcou[mais_marcou['Usuarios'] == 'Jessica da Silva '].index
mais_marcou = mais_marcou.drop(indexNames)
indexNames = mais_marcou[mais_marcou['Usuarios'] == 'CRISCIANE BARROS DE MOURA '].index
mais_marcou = mais_marcou.drop(indexNames)
indexNames = mais_marcou[mais_marcou['Usuarios'] == 'Gabriela'].index
mais_marcou = mais_marcou.drop(indexNames)

mais_marcou = mais_marcou.reset_index(drop=True)

mais_marcou = mais_marcou.head(3)

# RELATÓRIO ------------------------------------------------------------------------------

print('''
Unidade Gerson Schorr Centro\n
Relatório dia 10/05/2022, Terça-Feira



---------------------------- INDICADORES FINANCEIROS ----------------------------

Faturamento Total: R${:.2f} (NAO COLOCAR)

Ticket Médio Geral da Clinica: R${:.2f}

Faturamento Convenios R${:.2f} 

Convenios que mais geraram Faturamento:
    
    {}
    
Convenios mais atendidos:

    {}
    
** Ticket Médio dos convenios:

Faturamento Particular: R${:.2f}
- Cartão: R${:.2f}
- Dinheiro: R${:.2f}
    

-------------------- INDICADORES DE PRODUTIVIDADE DOS MÉDICOS -------------------


Quantidade de CONSULTAS por MÉDICO: 

{}

Quantidade de Cirurgias e Pequenos atendimentos por MÉDICO:

{}

FATURAMENTO por MÉDICO:

{}

** Ticket Medio dos Medicos (retirar guia orçamento)


------------------------------ INDICADORES DE AGENDA ----------------------------

Numero de Consultas: {:.0f} consultas

Numero aproximado de Primeiras vezes com base nas Triagens: {} ({:.0f}% das consultas)

Quantos pacientes estavam marcados: {:.0f} pacientes

Quantidade de faltas: {} ({:.0f}% da agenda)

Marcações na agenda (Telefone e Whatsapp): {}

Usuarios que mais marcaram:

{}


---------------------------------------------------------------------------------

'''.format(faturamento_bruto,ticket_medio,fat_convenios,faturamento_valores,cons_por_conv,total_caixa,total_cartao,total_din,medicos_consultas,sadts,fat_medicos,total_consultas,triagens,porcentriagem,total_Marcações,total_Faltas,porcent_faltas,qtd_consultas_marcadas,mais_marcou))



